# Scheduling

The scheduler is responsible for managing the entirety of the process life cycle and for selecting the next process to run. In the next few sections we will describe the process life cycle and several methods for ordering runnable processes. Before we discuss details we need to define a few new terms. Earlier we described ho we will approach virtualizing the CPU, by allowing a process to run for an amount of time and then switching for another process and then repeating these steps as long as there are processes that need to run. We allot a bounded amount of time for a process to run in a given turn and this bounded time is called a *time slice*. We need a mechanism for changing what the active process on a CPU is, we call this mechanism a *context switch*. With these definitions let's jump into the mechanism first.

## The Process Life Cycle

Processes are created in one of two ways: the operating system creates the first process (called init in UNIX-like systems) during boot up, or any process can use a system call (e.g., `fork()`, `clone()`, `NtCreateProcess()`, etc). Processes can also be terminated in three ways: by itself by calling `exit()`, `ExitProcess()`, or equivalent, due to an error (e.g., segmentation fault, division by zero, etc.), or by another process calling (`kill()`, `TerminateProcess()`, or similar).

Once created, a process has the following life cycle.
    
```{figure} ../images/scheduling/ProcessLifecycle.png
---
name: lifecycle
---
The process life cycle showing the possible transitions between the states.
```

As shown in {numref}`lifecycle`, there are 3 possible states for a process and all the valid stat transitions are represented by the arrows. A running process is one that is currently active on a CPU. A running process can move to either the 'Ready' or 'Blocked' state. To move to the Blocked state (labeled 1), a process must take an action that will require 'significant'. Some examples of blocking operations are: issuing a read from disk or from a network socket that does not have data ready, trying to acquire certain kinds of locks that are already owned by another process or thread, etc. A Running process that takes an action that will block immediately moves to the Blocked state, allowing another process to use the CPU while it waits To move to Ready from Running (labeled 2), the running process has to either yield the remainder of its allotted time slice, or be stopped by the OS after exhausting its slice. Moving from Ready to Running (labeled 3) requires that the OS to select the process as the next to execute and place it on a processor. The final transition we have not discussed is from Blocked to Ready (labeled 4), this transition happens when the blocking request is now complete or ready.

## Performing a Context Switch

[Earlier](cont:scheduling:process:abstraction) we described the data structures used to hold process information, now we need a way to change the active process on a CPU. When the OS decides to change the active process all of the state associated with the outgoing process is cached in its PCB. Next the OS selects a new process to take the CPU and loads the incoming process state from its PCB and sets the CPU to run the incoming state from where it last left off. Note that each of the state transitions shown in {numref}`lifecycle` are context switches. At each of these points an outgoing process is taken off a CPU and an incoming one is placed on it and resumed. This transition is an expensive one, even though we are saving all the state needed to resume a process, modern processors have several caches (data and instruction caches, the translation look aside buffer, etc.) used to help a process run more quickly that are tied to the executing process. When a new process is placed on a CPU, the contents of these caches are not related and either cleared or need to be replaced by the new process.

## Selecting the Next Process

With all the parts in place to move processes on and off a CPU we are only missing a method for choosing what the next process to run on a CPU will be. Selecting the next process requires that the OS developer consider a number of possible criterion to optimize and to balance against each other. We must weigh things like responsiveness, turnaround time, throughput, predictability, fairness, and load on other parts of the system in making this decision, so let's take a look at what each of these considerations are.

### Throughput

We define throughput as maximizing the number of processes completed per unit of time. To optimize for throughput we want to ensure that the active process stays on the CPU for as long as it isn't blocked.

### Turnaround Time

Turnaround time is the amount of time between a job submission and its termination. Minimizing this time often helps with throughput. A consequence of optimizing for throughput and turnaround time is that the CPU is kept busy.

### Responsiveness

Responsiveness is the amount of time required for the system to respond to an event or a request. If a user is interacting with the system, they will likely notice if the scheduler is not prioritizing responsiveness if they have to wait for key strokes or other input to have their effect. 